In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
import requests, sys, re

In [3]:
url_template = 'https://www.wunderground.com/history/daily/gb/london/EGLC/date/{}-{:02d}-{:02d}'

In [4]:
def get_weather_for_one_day(year, month, day):
    url = url_template.format(year, month, day)
    
    bi = FirefoxBinary(r'C:\Program Files\Mozilla Firefox\\firefox.exe')
    br = webdriver.Firefox(firefox_binary=bi)

# This starts an instance of Firefox at the specified URL:
    br.get(url)

# I understand that at this point the data is in html format and can be
# extracted with BeautifulSoup:
    sopa = BeautifulSoup(br.page_source, 'lxml')

# Close the firefox instance started before:
    br.quit()

# I'm only interested in the tables contained on the page:
    tablas = sopa.find_all('table')
    table=tablas[2]

    try:
        df = pd.read_html(str(table), na_values=['---'], header=0)[0]
    except Exception as e:
        print(e)
        raise ValueError('No data is available for {}/{:02d}/{:02d}'.format(year, month, day))
   # df.drop(df.shape[0] - 1, inplace=True)
   # df.columns = ['city', 'max_temp', 'min_temp', 'rain', 'wind', 'daylight']
    df['date'] = pd.to_datetime('{}/{}/{}'.format(year, month, day))
    
    return df

In [5]:
def get_weather_from_date(year, month, day):
    date = datetime(year, month, day)
    today = datetime(2017,9,30)
    df = []
    while (today - date).days >= 1:
        try:
            df.append(get_weather_for_one_day(date.year, date.month, date.day))
        except ValueError as e:
            print(e)
        finally:
            date += timedelta(days=1)
    return pd.concat(df)

In [8]:
df = get_weather_from_date(2017,8,1)

In [9]:
df.head(10)

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,date
0,11:20 PM,61 F,52 F,72 %,SW,8 mph,0 mph,29.90 in,0.0 in,Fair,2017-08-01
1,11:50 PM,61 F,52 F,72 %,SW,8 mph,0 mph,29.90 in,0.0 in,Fair,2017-08-01
2,12:20 AM,59 F,52 F,77 %,SW,9 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
3,12:50 AM,59 F,52 F,77 %,SW,6 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
4,1:20 AM,59 F,52 F,77 %,SW,7 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
5,1:50 AM,57 F,52 F,82 %,SW,5 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
6,2:20 AM,57 F,54 F,88 %,SW,6 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
7,2:50 AM,57 F,54 F,88 %,SW,6 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
8,3:20 AM,57 F,54 F,88 %,SW,5 mph,0 mph,29.93 in,0.0 in,Fair,2017-08-01
9,3:50 AM,57 F,54 F,88 %,SW,6 mph,0 mph,29.90 in,0.0 in,Fair,2017-08-01


In [10]:
df.tail()

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,date
43,8:50 PM,59 F,54 F,82 %,WSW,10 mph,23 mph,30.01 in,0.0 in,Partly Cloudy,2017-09-29
44,9:20 PM,59 F,54 F,82 %,WSW,8 mph,0 mph,30.01 in,0.0 in,Fair,2017-09-29
45,9:50 PM,59 F,54 F,82 %,WSW,8 mph,0 mph,30.01 in,0.0 in,Fair,2017-09-29
46,10:20 PM,57 F,54 F,88 %,WSW,7 mph,0 mph,30.01 in,0.0 in,Fair,2017-09-29
47,10:50 PM,57 F,54 F,88 %,SSW,6 mph,0 mph,30.01 in,0.0 in,Fair,2017-09-29


In [11]:
df.to_csv('londonweather.csv', index=False)